# Dynamic functional connectivity analysis

#### fMRI Data Barrow Neurological Institute
N subjects: 58 (29 autistic, 29 non-autistic). Sliding-window approach was used (window length 13TRs/39s; window step 1TR/3s) and 108 windows were obtained. 

This script: 
- Five states of dynamic functional connectivity: high negative connectivity, moderate negative, low-uncorrelated, moderate positive and high positive.
- Three connectivity state indices: Proportion of windows, Mean Dwell Time (MDT) and Probability of transition (PT).  

In [1]:
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import plotly.graph_objects as go

#Import stats from scipy library
from scipy import stats

#Read .csv file 
project = pd.read_csv('DynFC_BNI_raTPJ_rpTPJ.csv')

#Change column names for simplicity. Delete the first part of the name 'R2R target ROI mask_rvaTPJ : + mask_rvpTPJ @ rest x'
project.columns = [col.replace('R2R target ROI rpTPJ : + raTPJ @ rest x', '') for col in project.columns]

#load first five rows of the data set
project.head()

,FIQ,SRS_TOTAL_RAW,SRS_AWARENESS_RAW,SRS_COGNITION_RAW,SRS_COMMUNICATION_RAW,SRS_MOTIVATION_RAW,SRS_MANNERISMS_RAW,SRS_TOTAL_T,SRS_AWARENESS_T,SRS_COGNITION_T,...,Time99,Time100,Time101,Time102,Time103,Time104,Time105,Time106,Time107,Time108
0,131,122.0,12.0,28.0,38.0,19.0,25.0,79.0,66.0,86.0,...,0.579339,0.619573,0.605167,0.558422,0.554635,0.674923,0.916882,1.069563,1.275316,1.517891
1,110,82.0,7.0,14.0,31.0,11.0,19.0,65.0,52.0,62.0,...,0.537447,0.433278,0.259658,0.042305,-0.154327,-0.236219,-0.151669,-0.138327,-0.105466,-0.060308
2,117,60.0,6.0,7.0,26.0,12.0,9.0,57.0,49.0,49.0,...,-0.165314,-0.003568,0.168905,0.318829,0.460822,0.611519,0.759411,0.841748,0.932298,1.016136
3,114,58.0,3.0,14.0,18.0,15.0,8.0,56.0,41.0,62.0,...,0.675192,0.955481,1.075603,1.091327,1.094420,1.048578,0.885900,0.779036,0.597375,0.320921
4,109,146.0,19.0,26.0,50.0,23.0,28.0,87.0,86.0,83.0,...,1.117257,1.586249,2.119633,2.112867,1.560629,1.139224,0.969803,1.005435,1.067116,1.118772


In [ ]:
#Create a new data frame 'windows' with the last 108 columns of the 'project' data frame 
#i.e. only the z-scores per subject, per window (in pur case, 108 windows in total). 

windows = project[project.columns[-108:project.columns.size]]
project.head()

In [ ]:
#Select windows with high negative, moderate negative, low-uncorrelated, moderate positive and high postitive FC values. 

high_negative = windows<-0.5
moderate_negative = (windows >= -0.5) & (windows < -0.25)  
low_uncorrelated= (windows >= -0.25) & (windows <= 0.25) 
moderate_positive = (windows > 0.25) & (windows <= 0.5)
high_positive = windows>0.5


#Create three new columns with the proportion of windows in each state, per subject. 
prop_high_negative = high_negative.sum(axis = 1) / 108
prop_moderate_negative = moderate_negative.sum(axis = 1) / 108
prop_low_uncorrelated = low_uncorrelated.sum(axis = 1) / 108
prop_moderate_positive = moderate_positive.sum(axis = 1) / 108
prop_high_positive = high_positive.sum(axis = 1) / 108

project['prop_high_negative'] = prop_high_negative
project['prop_moderate_negative'] = prop_moderate_negative
project['prop_low_uncorrelated'] = prop_low_uncorrelated
project['prop_moderate_positive'] = prop_moderate_positive
project['prop_high_positive'] = prop_high_positive


project.head()

In [ ]:
#Create new data frames for each group, autism spectrum disorder (asd) and typically developing (td).

#New data frames
asd = project.loc[:28]
td = project.loc[29:]

In [ ]:
#Standard deviation (SD) for each group and state to add error bars in the next plot. 
asd_prop_high_negative_std = np.std(asd['prop_high_negative'])
asd_prop_moderate_negative_std = np.std(asd['prop_moderate_negative'])
asd_prop_low_uncorrelated_std = np.std(asd['prop_low_uncorrelated'])
asd_prop_moderate_positive_std = np.std(asd['prop_moderate_positive'])
asd_prop_high_positive_std = np.std(asd['prop_high_positive'])

asd_std = np.array([asd_prop_high_negative_std, asd_prop_moderate_negative_std, asd_prop_low_uncorrelated_std, asd_prop_moderate_positive_std, asd_prop_high_positive_std])
print(asd_std)

td_prop_high_negative_std = np.std(td['prop_high_negative'])
td_prop_moderate_negative_std = np.std(td['prop_moderate_negative'])
td_prop_low_uncorrelated_std = np.std(td['prop_low_uncorrelated'])
td_prop_moderate_positive_std = np.std(td['prop_moderate_positive'])
td_prop_high_positive_std = np.std(td['prop_high_positive'])

td_std = np.array([td_prop_high_negative_std, td_prop_moderate_negative_std, td_prop_low_uncorrelated_std, td_prop_moderate_positive_std, td_prop_high_positive_std])

print(td_std)

In [ ]:
#Interactive Proportion of windows plot with matplotlib

asd_prop_high_negative_mean = np.mean(asd['prop_high_negative'])
asd_prop_moderate_negative_mean = np.mean(asd['prop_moderate_negative'])
asd_prop_low_uncorrelated_mean = np.mean(asd['prop_low_uncorrelated'])
asd_prop_moderate_positive_mean = np.mean(asd['prop_moderate_positive'])
asd_prop_high_positive_mean = np.mean(asd['prop_high_positive'])
      
td_prop_high_negative_mean = np.mean(td['prop_high_negative'])
td_prop_moderate_negative_mean = np.mean(td['prop_moderate_negative'])
td_prop_low_uncorrelated_mean = np.mean(td['prop_low_uncorrelated'])
td_prop_moderate_positive_mean = np.mean(td['prop_moderate_positive'])
td_prop_high_positive_mean = np.mean(td['prop_high_positive'])

states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_prop_high_negative_mean, asd_prop_moderate_negative_mean, asd_prop_low_uncorrelated_mean, asd_prop_moderate_positive_mean, asd_prop_high_positive_mean], error_y=dict(type = "data", array = asd_std)),
    go.Bar(name='TD Group', x=states, y=[td_prop_high_negative_mean, td_prop_moderate_negative_mean, td_prop_low_uncorrelated_mean, td_prop_moderate_positive_mean, td_prop_high_positive_mean], error_y=dict(type = "data", array = td_std)),
])
# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Average proportion of windows in each state",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="Proportion of windows",)
fig.show()

-----------------------------------------
Next, we want to obtain Mean Dwell Time (MDT) and Probability of transition (PT). 

- MDT: average number of windows that were continuous on time and attributed to the same state.
- PT: number of transitions from other states to a specific one over total available transitions(number of state changes).

--------------------------------------------

In [ ]:
import itertools

MDT_high_negative = np.zeros(58)
MDT_moderate_negative = np.zeros(58)
MDT_low_uncorrelated = np.zeros(58)
MDT_moderate_positive = np.zeros(58)
MDT_high_positive = np.zeros(58)

PT_high_negative = np.zeros(58)
PT_moderate_negative = np.zeros(58)
PT_low_uncorrelated = np.zeros(58)
PT_moderate_positive = np.zeros(58)
PT_high_positive = np.zeros(58)

for i in np.arange(58):
    high_negative_condition = high_negative.loc[i]
    condition1 = high_negative_condition
    high_negative_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition1 ) if key]
    MDT1 = np.mean(high_negative_consecutive_count)
    MDT_high_negative[i] = MDT1
    size_high_negative = len(high_negative_consecutive_count) 
    
    moderate_negative_condition = moderate_negative.loc[i]
    condition2 = moderate_negative_condition
    moderate_negative_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition2 ) if key]
    MDT2 = np.mean(moderate_negative_consecutive_count)
    MDT_moderate_negative[i] = MDT2
    size_moderate_negative = len(moderate_negative_consecutive_count) 
    
    low_uncorrelated_condition = low_uncorrelated.loc[i]
    condition3 = low_uncorrelated_condition
    low_uncorrelated_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition3 ) if key]
    MDT3 = np.mean(low_uncorrelated_consecutive_count)
    MDT_low_uncorrelated[i] = MDT3
    size_low_uncorrelated = len(low_uncorrelated_consecutive_count) 
    
    moderate_positive_condition = moderate_positive.loc[i]
    condition4 = moderate_positive_condition
    moderate_positive_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition4 ) if key]
    MDT4 = np.mean(moderate_positive_consecutive_count)
    MDT_moderate_positive[i] = MDT4
    size_moderate_positive = len(moderate_positive_consecutive_count)
    
    high_positive_condition = high_positive.loc[i]
    condition5 = high_positive_condition
    high_positive_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition5 ) if key]
    MDT5 = np.mean(high_positive_consecutive_count)
    MDT_high_positive[i] = MDT5
    size_high_positive = len(high_positive_consecutive_count)
    
    
    total_changes = (size_high_negative + size_moderate_negative + size_low_uncorrelated + size_moderate_positive + size_high_positive) - 1
    
    PT_high_negative[i] = (size_high_negative / total_changes)
    PT_moderate_negative[i] = (size_moderate_negative / total_changes)
    PT_low_uncorrelated[i] = (size_low_uncorrelated / total_changes)
    PT_moderate_positive[i] = (size_moderate_positive / total_changes)
    PT_high_positive[i] = (size_high_positive / total_changes)
    
    
print(MDT_high_negative)
print(MDT_moderate_negative)
print(MDT_low_uncorrelated)
print(MDT_moderate_positive)
print(MDT_high_positive)
print()
print(PT_high_negative)
print(PT_moderate_negative)
print(PT_low_uncorrelated)
print(PT_moderate_positive)
print(PT_high_positive)

In [ ]:
#Create new columns in the project data frame with the new indices that we have obtained for each state

project['MDT_high_negative'] = MDT_high_negative
project['MDT_moderate_negative'] = MDT_moderate_negative
project['MDT_low_uncorrelated'] = MDT_low_uncorrelated
project['MDT_moderate_positive'] = MDT_moderate_positive
project['MDT_high_positive'] = MDT_high_positive

project['PT_high_negative'] = PT_high_negative
project['PT_moderate_negative'] = PT_moderate_negative
project['PT_low_uncorrelated'] = PT_low_uncorrelated
project['PT_moderate_positive'] = PT_moderate_positive
project['PT_high_positive'] = PT_high_positive

project.head()

In [ ]:
#Redefine td and asd so they have the new columns added to the 'project' data frame. 
asd = project.loc[:28]
td = project.loc[29:] 

asd.replace(np.nan, 0, inplace = True)
td.replace(np.nan, 0, inplace = True)

In [ ]:
#Standard deviation (SD) for each group and state to add error bars in the next plot. 

asd_MDT_high_negative_std = np.std(asd['MDT_high_negative'])
asd_MDT_moderate_negative_std = np.std(asd['MDT_moderate_negative'])
asd_MDT_low_uncorrelated_std = np.std(asd['MDT_low_uncorrelated'])
asd_MDT_moderate_positive_std = np.std(asd['MDT_moderate_positive'])
asd_MDT_high_positive_std = np.std(asd['MDT_high_positive'])

asd_std_MDT = np.array([asd_MDT_high_negative_std, asd_MDT_moderate_negative_std, asd_MDT_low_uncorrelated_std, asd_MDT_moderate_positive_std, asd_MDT_high_positive_std])
print(asd_std_MDT)

td_MDT_high_negative_std = np.std(td['MDT_high_negative'])
td_MDT_moderate_negative_std = np.std(td['MDT_moderate_negative'])
td_MDT_low_uncorrelated_std = np.std(td['MDT_low_uncorrelated'])
td_MDT_moderate_positive_std = np.std(td['MDT_moderate_positive'])
td_MDT_high_positive_std = np.std(td['MDT_high_positive'])

td_std_MDT = np.array([td_MDT_high_negative_std, td_MDT_moderate_negative_std, td_MDT_low_uncorrelated_std, td_MDT_moderate_positive_std, td_MDT_high_positive_std])

print(td_std_MDT)

In [ ]:
#Interactive Mean Dwell Time plot

asd_MDT_high_negative_mean = np.mean(asd['MDT_high_negative'])
asd_MDT_moderate_negative_mean = np.mean(asd['MDT_moderate_negative'])
asd_MDT_low_uncorrelated_mean = np.mean(asd['MDT_low_uncorrelated'])
asd_MDT_moderate_positive_mean = np.mean(asd['MDT_moderate_positive'])
asd_MDT_high_positive_mean = np.mean(asd['MDT_high_positive'])
      
td_MDT_high_negative_mean = np.mean(td['MDT_high_negative'])
td_MDT_moderate_negative_mean = np.mean(td['MDT_moderate_negative'])
td_MDT_low_uncorrelated_mean = np.mean(td['MDT_low_uncorrelated'])
td_MDT_moderate_positive_mean = np.mean(td['MDT_moderate_positive'])
td_MDT_high_positive_mean = np.mean(td['MDT_high_positive'])



states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_MDT_high_negative_mean, asd_MDT_moderate_negative_mean, asd_MDT_low_uncorrelated_mean, asd_MDT_moderate_positive_mean, asd_MDT_high_positive_mean], error_y=dict(type = "data", array = asd_std_MDT)),
    go.Bar(name='TD Group', x=states, y=[td_MDT_high_negative_mean, td_MDT_moderate_negative_mean, td_MDT_low_uncorrelated_mean, td_MDT_moderate_positive_mean, td_MDT_high_positive_mean], error_y=dict(type = "data", array = td_std_MDT)),
])
# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Mean Dwell Time",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="MDT",)
fig.show()

In [ ]:
#Standard deviation (SD) for each group and state to add error bars in the next plot. 
asd_PT_high_negative_std = np.std(asd['PT_high_negative'])
asd_PT_moderate_negative_std = np.std(asd['PT_moderate_negative'])
asd_PT_low_uncorrelated_std = np.std(asd['PT_low_uncorrelated'])
asd_PT_moderate_positive_std = np.std(asd['PT_moderate_positive'])
asd_PT_high_positive_std = np.std(asd['PT_high_positive'])

asd_std_PT = np.array([asd_PT_high_negative_std, asd_PT_moderate_negative_std, asd_PT_low_uncorrelated_std, asd_PT_moderate_positive_std, asd_PT_high_positive_std])
print(asd_std_PT)

td_PT_high_negative_std = np.std(td['PT_high_negative'])
td_PT_moderate_negative_std = np.std(td['PT_moderate_negative'])
td_PT_low_uncorrelated_std = np.std(td['PT_low_uncorrelated'])
td_PT_moderate_positive_std = np.std(td['PT_moderate_positive'])
td_PT_high_positive_std = np.std(td['PT_high_positive'])

td_std_PT = np.array([td_PT_high_negative_std, td_PT_moderate_negative_std, td_PT_low_uncorrelated_std, td_PT_moderate_positive_std, td_PT_high_positive_std])

print(td_std_PT)

In [ ]:
#Interactive Probability of Transition plot

asd_PT_high_negative_mean = np.mean(asd['PT_high_negative'])
asd_PT_moderate_negative_mean = np.mean(asd['PT_moderate_negative'])
asd_PT_low_uncorrelated_mean = np.mean(asd['PT_low_uncorrelated'])
asd_PT_moderate_positive_mean = np.mean(asd['PT_moderate_positive'])
asd_PT_high_positive_mean = np.mean(asd['PT_high_positive'])
      
td_PT_high_negative_mean = np.mean(td['PT_high_negative'])
td_PT_moderate_negative_mean = np.mean(td['PT_moderate_negative'])
td_PT_low_uncorrelated_mean = np.mean(td['PT_low_uncorrelated'])
td_PT_moderate_positive_mean = np.mean(td['PT_moderate_positive'])
td_PT_high_positive_mean = np.mean(td['PT_high_positive'])

states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_PT_high_negative_mean, asd_PT_moderate_negative_mean, asd_PT_low_uncorrelated_mean, asd_PT_moderate_positive_mean, asd_PT_high_positive_mean], error_y=dict(type = "data", array = asd_std_PT)),
    go.Bar(name='TD Group', x=states, y=[td_PT_high_negative_mean, td_PT_moderate_negative_mean, td_PT_low_uncorrelated_mean, td_PT_moderate_positive_mean, td_PT_high_positive_mean], error_y=dict(type = "data", array = td_std_PT)),
])
# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Probability of Transition",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="PT",)
fig.show()

In [ ]:
#project.to_csv('processed_BNI_1_raTPJ_rpTPJ_39s_001_008.csv')